# About

This notebook uses canopy height data for the state of California obtained through the California Forest Observatory (CFO) API to create a canopy height raster layer for Santa Barbara County. See more about CFO's API here:
https://github.com/forestobservatory/cfo-api

**NOTEBOOK VARIABLES**: 
- `year` (int): year from which we will access the CFO California canopy height raster. Must be 2016, 2018, or 2020 since CFO's canopy height data is only available for those years.
- file path to which the Santa Barbara County subset of the canopy height data will be saved. 

**OUTPUT**:
-  canopy height raster layer for Santa Barbara County in the year *y*



A CFO account is needed to run the notebook. 

An alternative way of doing this could be to follow the 'Custom Data Clipping' section in this tutorial. 
https://github.com/forestobservatory/cfo-api/blob/master/demos/api-introduction.ipynb
To import `gdal` to the notebook, add `from osgeo import gdal`. 

# Install California Forest Observatory API package if not available

In [4]:
#pip install cfo

  Using cached cfo-1.0.1-py3-none-any.whl (28 kB)
  Using cached retrying-1.3.3-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [17]:
import os
import pandas as pd
import geopandas as gpd
import rioxarray as riox
from shapely.geometry import box

import json
import rasterio
from rasterio.mask import mask

import matplotlib.pyplot as plt

import cfo
from osgeo import gdal

# Specify year and file path for canopy height data

In [2]:
# ***************************************************
# ************* NOTEBOOK VARIABLES ******************

# year shuold be 2016, 2018 or 2020
year = 2020


# ***************************************************
# ***************************************************

# Authenticate 

In [3]:
forest_api = cfo.api()
forest_api.authenticategalaz-garcia@nceas.ucsb.edu()

CFO E-mail:  galaz-garcia@nceas.ucsb.edu
CFO Password:  ···········


200

# Download and open CA canopy height raster

In [12]:
# find item id
ca_canopy_itemid = forest_api.search(geography='California', metric='CanopyHeight', year=year)[0]
print(ca_canopy_itemid)
forest_api.search(geography='California', metric='CanopyHeight', year=2020)

['California-Vegetation-CanopyHeight-2020-Summer-00010m',
 'California-Vegetation-CanopyHeight-2020-Fall-00003m']

In [6]:
# filepath to temporary folder
ca_canopy_fp = os.path.join(os.getcwd(), 
                            'temp',
                            'ca_canopyheight_'+str(year)+'.tif')

In [7]:
# download (takes a couple of minutes)
forest_api.download(ca_canopy_itemid, ca_canopy_fp)

# open raster reader
ca_canopy = rasterio.open(ca_canopy_fp)

# Get bounding box for Santa Barbara County mainland

In [8]:
# https://automating-gis-processes.github.io/CSC18/lessons/L6/clipping-raster.html
def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    return [json.loads(gdf.to_json())['features'][0]['geometry']]


In [9]:
# path to SB county shapefile
sb_fp = '/home/jovyan/msai4earth-esa/shapefiles_n_rasters/SB_geometry/SB_only.shp'

# read as GeoDataFrame
sb = gpd.read_file(sb_fp)

# match CRS with ca_canopy raster
sb = sb.to_crs(ca_canopy.crs)

# we will only use the mainland for the analysis
sb_mainland_bounds = sb.bounds.iloc[0]

bbox = box(sb_mainland_bounds.minx,
           sb_mainland_bounds.miny,
           sb_mainland_bounds.maxx,
           sb_mainland_bounds.maxy)

bbox_df = gpd.GeoDataFrame({'geometry':bbox}, index=[0], crs = sb.crs)
coords = getFeatures(bbox_df)

# Mask CA canopy height raster with Santa Barbara county's bounding box

In [10]:
out_img, out_transform = mask(dataset=ca_canopy, shapes=coords, crop=True)
out_img = out_img.squeeze()

# Save Santa Barbara County canopy height raster

In [11]:
# Create directory if needed
sb_lidar = os.path.join(os.getcwd(), 
                  'SantaBarbaraCounty_lidar')
if not os.path.exists(sb_lidar):
    os.makedirs(sb_lidar)
    
# file path where raster will be saved
fp = os.path.join(os.getcwd(),
                  'SantaBarbaraCounty_lidar',
                  'SantaBarbaraCounty_lidar_'+str(year)+'.tif')
# save raster
with rasterio.open(
        fp,  # file path
        'w',           # w = write
        driver='GTiff', # format
        height = out_img.shape[0], 
        width = out_img.shape[1],
        count = 1,  # number of raster bands in the dataset
        dtype = 'int16',
        crs = sb.crs,
        transform = out_transform,
) as dst:
    dst.write(out_img, 1)

# Delete CA canopy height raster

In [12]:
os.remove(ca_canopy_fp)

# Open SB canopy height raster to check shape

In [ ]:
clip = rasterio.open(fp).read([1]).squeeze()

fig, ax = plt.subplots(figsize=(15, 15))
ax.imshow(clip)
plt.show()

# Clipping through gdal

In [5]:
sb_fp = '/home/jovyan/msai4earth-esa/shapefiles_n_rasters/SB_geometry/SB_only.shp'
shp_file = gpd.read_file(sb_fp)
shp_file.to_file(os.path.join(os.getcwd(),'temp','SB_only.geojson'), driver='GeoJSON')

/srv/conda/envs/notebook/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [13]:
vector = os.path.join(os.getcwd(),'temp','SB_only.geojson')

# get the 2020 10m canopy height asset id
czu_search = forest_api.search(geography='California', metric='CanopyHeight', year=year, resolution=10)
czu = czu_search[0]

# and get the gdal file path reference
input_file = forest_api.fetch(czu, gdal=True)
print(f"We'll download from: {input_file}")

We'll download from: /vsigs/cfo-public/vegetation/California-Vegetation-CanopyHeight-2020-Summer-00010m.tif


In [18]:
# set the output raster file
output_file = os.path.join(os.getcwd(),'SantaBarbaraCounty_lidar', "SB-perimeter-CanopyHeight-2020.tif")

# set the gdalwarp options
options = gdal.WarpOptions(
    creationOptions = ["COMPRESS=DEFLATE", "TILED=YES", "BIGTIFF=YES", "NUM_THREADS=ALL_CPUS"],
    cutlineDSName = vector,
    cropToCutline = True
)

# and run the command
warp = gdal.Warp(output_file, input_file, options=options)
warp.FlushCache()